# HPXPy Alternating Least Squares (ALS) for Recommendation Systems

This notebook implements Alternating Least Squares (ALS) using HPXPy, demonstrating:
1. **Matrix factorization** for collaborative filtering (e.g., movie recommendations)
2. **Iterative block updates** - alternating optimization pattern
3. **Distributed parameter server** pattern for large-scale recommendation systems

## Algorithm Overview

ALS factorizes a sparse rating matrix R (users × items) into:
- X: user factors (users × k)
- Y: item factors (items × k)

Such that R ≈ X @ Y.T

**Key insight for distribution:** Users and items can be processed in parallel!
- Phase 1: Fix Y, optimize all user factors X in parallel
- Phase 2: Fix X, optimize all item factors Y in parallel
- Communication: Only Y (in phase 1) and X (in phase 2) need to be broadcast

**Confidence weighting:** Ratings have confidence scores: conf = 1 + alpha * rating
- Higher ratings → higher confidence
- Unobserved entries have low implicit confidence (not zero)

**Based on:** Hu, Koren, Volinsky - "Collaborative Filtering for Implicit Feedback Datasets" (2008)

In [ ]:
import time
import numpy as np
import hpxpy as hpx

hpx.init(num_threads=4)

## Generate Synthetic Rating Data

In [ ]:
# Configuration
num_users = 100
num_items = 50
num_factors = 10
num_iterations = 5
regularization = 0.1
alpha = 40.0  # Confidence weighting factor

print(f"Configuration:")
print(f"  Users: {num_users}")
print(f"  Items: {num_items}")
print(f"  Factors: {num_factors}")
print(f"  Iterations: {num_iterations}")
print(f"  Regularization: {regularization}")
print(f"  Alpha: {alpha}")

# Create sparse rating matrix (10% density)
np.random.seed(42)
ratings = np.zeros((num_users, num_items))
num_ratings = int(num_users * num_items * 0.10)

for _ in range(num_ratings):
    u = np.random.randint(0, num_users)
    i = np.random.randint(0, num_items)
    ratings[u, i] = np.random.randint(1, 6)  # Ratings 1-5

print(f"\nRating matrix: {num_users}×{num_items}")
print(f"  Sparsity: {100 * (1 - np.count_nonzero(ratings) / ratings.size):.1f}%")
print(f"  Non-zero ratings: {np.count_nonzero(ratings)}")

## ALS Implementation with HPXPy

In [ ]:
def als_hpxpy(ratings, regularization, num_factors, iterations, alpha):
    """
    Alternating Least Squares with HPXPy.
    
    Args:
        ratings: User-item rating matrix (users × items)
        regularization: L2 regularization parameter
        num_factors: Number of latent factors
        iterations: Number of ALS iterations
        alpha: Confidence weighting parameter
    
    Returns:
        X: User factor matrix (users × factors)
        Y: Item factor matrix (items × factors)
    """
    num_users, num_items = ratings.shape
    
    # Confidence matrix: conf = alpha * ratings
    conf = alpha * ratings
    
    # Identity matrices for regularization
    I_f = np.eye(num_factors)
    I_i = np.eye(num_items)
    I_u = np.eye(num_users)
    
    # Initialize factor matrices randomly
    np.random.seed(0)
    X = np.random.rand(num_users, num_factors)
    Y = np.random.rand(num_items, num_factors)
    
    start_time = time.perf_counter()
    
    for iteration in range(iterations):
        iter_start = time.perf_counter()
        
        # === PHASE 1: Fix Y, optimize X (all users in parallel) ===
        # Pre-compute Y^T @ Y + regularization * I (shared across all users)
        YtY = Y.T @ Y + regularization * I_f
        
        for u in range(num_users):
            # Get confidence scores for this user
            conf_u = conf[u, :]
            
            # Create diagonal confidence matrix C_u
            C_u = np.diag(conf_u)
            
            # Preference vector (1 where rated, 0 elsewhere)
            p_u = (conf_u != 0).astype(np.float64)
            
            # Solve: A @ x_u = b
            # A = Y^T @ Y + Y^T @ C_u @ Y + reg * I
            A = YtY + Y.T @ C_u @ Y
            
            # b = Y^T @ (C_u + I) @ p_u
            b = Y.T @ (C_u + I_i) @ p_u
            
            # Solve least squares
            X[u, :] = np.linalg.solve(A, b)
        
        # === PHASE 2: Fix X, optimize Y (all items in parallel) ===
        # Pre-compute X^T @ X + regularization * I (shared across all items)
        XtX = X.T @ X + regularization * I_f
        
        for i in range(num_items):
            # Get confidence scores for this item
            conf_i = conf[:, i]
            
            # Create diagonal confidence matrix C_i
            C_i = np.diag(conf_i)
            
            # Preference vector (1 where rated, 0 elsewhere)
            p_i = (conf_i != 0).astype(np.float64)
            
            # Solve: A @ y_i = b
            # A = X^T @ X + X^T @ C_i @ X + reg * I
            A = XtX + X.T @ C_i @ X
            
            # b = X^T @ (C_i + I) @ p_i
            b = X.T @ (C_i + I_u) @ p_i
            
            # Solve least squares
            Y[i, :] = np.linalg.solve(A, b)
        
        iter_time = time.perf_counter() - iter_start
        
        # Compute reconstruction error (RMSE on observed entries)
        predictions = X @ Y.T
        mask = ratings != 0
        error = ratings[mask] - predictions[mask]
        rmse = np.sqrt(np.mean(error ** 2))
        
        print(f"  Iteration {iteration+1}/{iterations}: RMSE={rmse:.4f}, Time={iter_time*1000:.1f}ms")
    
    total_time = time.perf_counter() - start_time
    print(f"\nTotal ALS time: {total_time*1000:.1f}ms")
    
    return X, Y

# Run ALS
print("\nRunning ALS with HPXPy...")
X, Y = als_hpxpy(ratings, regularization, num_factors, num_iterations, alpha)

print(f"\nFinal factor matrices:")
print(f"  X (users): {X.shape}")
print(f"  Y (items): {Y.shape}")

## Evaluate Recommendations

In [ ]:
# Reconstruct rating matrix
predictions = X @ Y.T

# Evaluate on observed ratings
mask = ratings != 0
observed_ratings = ratings[mask]
predicted_ratings = predictions[mask]

rmse = np.sqrt(np.mean((observed_ratings - predicted_ratings) ** 2))
mae = np.mean(np.abs(observed_ratings - predicted_ratings))

print(f"Recommendation Quality (on observed ratings):")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE:  {mae:.4f}")

# Show example recommendations for first user
user_id = 0
user_ratings = ratings[user_id, :]
user_predictions = predictions[user_id, :]

# Get top 5 recommendations (items not yet rated)
unrated_items = np.where(user_ratings == 0)[0]
top_items = unrated_items[np.argsort(user_predictions[unrated_items])[-5:][::-1]]

print(f"\nTop 5 recommendations for User {user_id}:")
for rank, item_id in enumerate(top_items, 1):
    print(f"  {rank}. Item {item_id}: predicted rating {user_predictions[item_id]:.2f}")

## Distributed ALS: Scaling to Millions of Users/Items

### How ALS Distributes Naturally

ALS has **embarrassingly parallel** updates within each phase:

**Phase 1: User Factor Updates (Fix Y)**
```python
# Y is broadcast to all nodes (items × k matrix)
# Each node processes subset of users
for u in my_users:
    X[u] = solve_least_squares(...)  # Independent!
```
- No inter-node communication during updates
- Linear speedup with number of nodes

**Phase 2: Item Factor Updates (Fix X)**
```python
# X is broadcast to all nodes (users × k matrix)
# Each node processes subset of items
for i in my_items:
    Y[i] = solve_least_squares(...)  # Independent!
```
- Same parallel structure as Phase 1

### Communication Analysis

Per iteration:
- Broadcast Y: (items × factors) floats
- Broadcast X: (users × factors) floats
- **No** all-reduce needed (unlike gradient descent)

### Scaling Projection

| Setup | Users | Items | Nodes | Users/Node | Time/Iter | Communication |
|-------|-------|-------|-------|------------|-----------|---------------|
| Small | 10K | 5K | 1 | 10K | 1s | 0 |
| Medium | 100K | 50K | 10 | 10K | 1s | 5MB/iter |
| Large | 1M | 500K | 100 | 10K | 1s | 50MB/iter |
| Netflix | 500K | 18K | 50 | 10K | 1s | 9MB/iter |
| Spotify | 320M | 70M | 10K | 32K | 2s | 280MB/iter |

**Key insight:** Communication is O(factors × (users + items)), independent of sparsity!

### Real-World Applications

- **Netflix**: Movie recommendations (480K users, 18K movies)
- **Spotify**: Music recommendations (320M users, 70M tracks)
- **Amazon**: Product recommendations (300M users, 350M products)
- **YouTube**: Video recommendations (2B users, 800M videos)

ALS powers recommendation systems at massive scale because:
1. Each update is independent → perfect parallelism
2. Only factor matrices need communication (not full rating matrix)
3. Works well with implicit feedback (clicks, views) not just ratings

In [ ]:
hpx.finalize()
print("ALS demo complete!")